<a href="https://colab.research.google.com/github/LordLean/Tracking-Sources-Of-Online-Disinformation/blob/main/twitter_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tweepy as tw
import pandas as pd

In [2]:
consumer_key= 'YgYPLTWSJtexC3VmMjsleDC6v'
consumer_secret= 'FfRodDZdTVDCTgXqZZ7kk7Vb20HWYNEAuq2dKtA4BlaToWrlB8'
access_token= '1317451540471447553-qqtVFh59xxjerW6mTjhrmue5LdSVAb'
access_token_secret= 'SKYvUevuYjQOf9yhdZvNOTJvxPFodDg7vWSlLNzn6AN26'

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [45]:
# Function created to extract coordinates from tweet if it has coordinate info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_coordinates(row):
    if row['Tweet Coordinates']:
        return row['Tweet Coordinates']['coordinates']
    else:
        return None

        
# Function created to extract place such as city, state or country from tweet if it has place info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_place(row):
    if row['Place Info']:
        return row['Place Info'].full_name
    else:
        return None


def get_untruncated_tweet(id):
  status = api.get_status(id, tweet_mode="extended")
  try:
      return status.retweeted_status.full_text
  except AttributeError:  # Not a Retweet
      return status.full_text

In [ ]:
search_words = "#disinformation -filter:retweets"
# date_since = "2018-11-16"
username = "frasepr"
max_tweets = 150

In [52]:
# User Timeline
# tweets = tw.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(max_tweets)

# API Search
tweets = tw.Cursor(api.search, q=search_words, 
                   lang="en",
                   tweet_mode='extended'
                   ).items(max_tweets)
 
# Pulling information from tweets iterable object
# Add or remove tweet information you want in the below list comprehension
tweets_list = [[tweet.full_text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang, tweet.source, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id_str, tweet.is_quote_status] for tweet in tweets]
 
# Creation of dataframe from tweets_list
# Add or remove columns as you remove tweet information
tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', 'Tweet Coordinates', 'Place Info', 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id', 'Replied Tweet User Id Str', 'Quote Status Bool'])
 
# Checks if there are coordinates attached to tweets, if so extracts them
tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
 
# Checks if there is place information available, if so extracts them
tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)

In [53]:
tweets_df

,Tweet Text,Tweet Datetime,Tweet Id,Twitter @ Name,Tweet Coordinates,Place Info,Retweets,Favorites,Language,Source,Replied Tweet Id,Replied Tweet User Id Str,Quote Status Bool
0,#Inkl #FarRight groups move to #Messaging apps...,2021-01-22 15:41:07,1352642457796808713,Rogeliketweety,None,None,0,0,en,Twitter for Android,None,None,False
1,Learn how we can serve your needs in the field...,2021-01-22 15:39:15,1352641987472744448,ScienceOfPCVE,None,None,1,0,en,Twitter Web App,None,None,False
2,@laurenboebert You are spreading #disinformati...,2021-01-22 15:35:17,1352640988607508481,SudoTruth,None,None,0,0,en,Twitter Web App,1352599661266268160,1201670995435646976,False
3,@digiphile @KABGreek Hey thanks @digiphile. \n...,2021-01-22 15:25:59,1352638648756432898,GrahamBrookie,None,None,0,3,en,Twitter Web App,1352635727398522882,1175221,False
4,Media Literacy Standards to Counter Truth Deca...,2021-01-22 15:07:36,1352634024720805888,Chahali,None,None,0,1,en,IFTTT,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1. #DataScience + #BigData visualization to id...,2021-01-21 19:22:50,1352335864895045632,UNICRI,None,None,4,3,en,Twitter Web App,1352335859689914369,56975985,False
146,"#Disinformation Holds Western Powers Back fr, ...",2021-01-21 19:01:58,1352330615027359748,VahidMokhtarip1,None,None,0,0,en,Twitter Web App,None,None,False
147,This #Qanus bullshitter biatch sits in the blo...,2021-01-21 18:59:58,1352330114038693890,JaanaNystrom,None,None,0,0,en,Twitter Web App,None,None,True
148,"@NPR ""said to be"" is amplification of rumor, n...",2021-01-21 18:51:22,1352327948162895875,ShapingYouth,None,None,0,1,en,Twitter Web App,1352283690861211650,5392522,True


In [54]:
tweet_index = 0
# [:,2] selects Tweet Id df column.
id = int(tweets_df.iloc[:,2][tweet_index])
get_untruncated_tweet(id)

'#Inkl #FarRight groups move to #Messaging apps as #Tech companies crack down on extremist #socialmedia - @conversationedu via @inkl https://t.co/PZLYAxG0e5 #AltRight #Disinformation #Misinformation #Politics #Tech #socialmedia #Chat #Guide #HowTo'

In [ ]:
# Redundant 

In [ ]:
# Collect tweets
tweets = tw.Cursor(api.search,
                       q=search_words,
                       lang="en"
                       ).items(10000)

# Collect a list of tweets
all_tweets = [tweet.user for tweet in tweets]

In [ ]:
from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim()

coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(tweets.location):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass